In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile
from glob import glob
from pathlib import Path
import os
from tqdm import tqdm

Loading spectral patterns

In [ ]:
channels=4 #fill up here the number of channels
folder_calibration = ...

species = np.array([1,2,3,4])
#species = np.array([1,2]) #for GFP and Alexa488
spectralpatterns=np.zeros((221,channels,species.shape[0]))
for i in species:
    filepath = Path(folder_calibration) / f'species{i}_medfilt.npy'
    with open(filepath, 'rb') as f2:
        avgspectrum_i=np.load(f2)
    if i==2:
        spectralpatterns[:,:,i-1]=avgspectrum_i[:,:]
    else:
        spectralpatterns[:,:,i-1]=avgspectrum_i[:,:]

Create folder structure and get list of samples

In [ ]:
folder_data =...
os.mkdir(os.path.join(folder_data, "filt"))

samples=[]
paths = sorted(glob(rf'{folder_data}/*.tif'))
for path in paths :
    samples.append(Path(path).stem)


Filtering

In [ ]:
for ind in tqdm(samples):
    img = tifffile.imread(Path(folder_data) / f'{ind}.tif')
    img= np.array(img)

    image_filtered=(np.zeros_like(img)).astype(np.int16)

    for z in range(img.shape[0]):
        Iavg_channels=np.mean(img[z,:,:,:],axis=(1,2))
        D=np.diag(1/Iavg_channels)
        specpatterns_z=np.zeros((species.shape[0],img.shape[1]))
        for i in species:
            specpatterns_z[i-1,:]=spectralpatterns[z,:,i-1]
        w=np.linalg.inv(specpatterns_z@D@np.transpose(specpatterns_z))@specpatterns_z@D
        for i in species:

            image_filtered[z,i-1,:,:]=sum([w[i-1,j]*img[z,j,:,:] for j in range(0,channels)])

    image_filtered[image_filtered<0]=0
    image_filtered=image_filtered.astype(np.uint16)
    tifffile.imwrite(Path(folder_data) / f'filt/{ind}_filt.tif', image_filtered,imagej=True)

Plots

In [ ]:
z_to_plot=50
for ind in tqdm(samples):
    img = tifffile.imread(Path(folder_data) / f'{ind}.tif')
    fout, axarrout = plt.subplots(1,channels)            

    for i in range(channels):
        axarrout[i].imshow(img[z_to_plot,i,:,:])
        print(np.mean(img[z_to_plot,i,:,:]))
    fout2, axarrout2 = plt.subplots(1,channels)

    for i in range(channels):
        axarrout2[i].imshow(image_filtered[z_to_plot,i,:,:])
        print(np.mean(image_filtered[z_to_plot,i,:,:]))